### Установка модели

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab and Kaggle notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Загрузка Unsloth модели

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.3: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Lora Adapter

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Загрузка модели!!!

In [ ]:
!unzip lora_model.zip

Archive:  lora_model.zip
   creating: lora_model/
  inflating: lora_model/README.md    
  inflating: lora_model/special_tokens_map.json  
  inflating: lora_model/adapter_model.safetensors  
  inflating: lora_model/tokenizer_config.json  
  inflating: lora_model/adapter_config.json  
  inflating: lora_model/tokenizer.json  


In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth 2025.3.3: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


<a name="Data"></a>
### Data Processing
Данные должны соответствовать такому формату для корректной работы:

```
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What's the capital of France?<|im_end|>
<|im_start|>assistant
Paris.<|im_end|>
```


In [ ]:
import json
from func_to_call import parse_all_data, parse_data_with_time


data_v1 = parse_all_data('datasets/train_set.json')

data_v2 = parse_data_with_time('datasets/val_set.json')

with open('parsed_tuning.json', 'w', encoding='utf-8') as f:
    json.dump(data_v1, f, ensure_ascii=False)

with open('parsed_dash.json', 'w', encoding='utf-8') as f:
    json.dump(data_v2, f, ensure_ascii=False)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd0 in position 9437183: unexpected end of data

In [ ]:
with open('parsed_tuning.json', 'r', encoding='utf-8') as f:
    training_data = json.load(f)

In [ ]:
system_prompt = '''Ты - система информационного поиска Национального исследовательского университета «Высшая школа экономики» (НИУ ВШЭ). Тебе дан вопрос и релевантные отрывки текста из нескольких документов.
Создай краткий и информативный ответ на заданный вопрос. Ты должна использовать только информацию из приведенных отрывков.
Используй непредвзятый и журналистский тон. Не повторяй текст. Не пытайся придумать ответ.
Если в контексте присутствуют ссылки на документы, выведи ответ со ссылками на источники.
Создай окончательный ответ ("FINAL ANSWER").
Отвечай только на русском языке за исключением специальных терминов.
ПРИМЕР:
QUESTION: Что такое ИУП?
=========
Content 1: Индивидуальный учебный план (ИУП) — документ студента, в соответствии с которым он обязуется освоить элементы обучения (дисциплин/элементов практической подготовки) в определенный период.
По каждому элементу ИУПа хранится информация
Content 2: период.
По каждому элементу ИУПа хранится информация о технологиях обучения, сроках изучения и прохождения промежуточной аттестации, объеме учебной нагрузки и основании включения этого элемента в учебный план.
Для студентов большинства
Content 3: план.
Для студентов большинства образовательных программ, ИУП составляется на полугодие, и все элементы, включенные в ИУП, обязательны для освоения в этот период.
Как ознакомиться со своим ИУП
Content 4: со своим ИУП. Ваш ИУП доступен в [электронной зачетке]( https://lms.hse.ru/index.php?page=gradebook) LMS НИУ ВШЭ. \nПодробнее об ИУП: https://www.hse.ru/studyspravka/plan
=========
FINAL ANSWER: Индивидуальный учебный план (ИУП) — документ студента, в соответствии с которым он обязуется освоить элементы обучения (дисциплин/элементов практической подготовки) в определенный период. \nИУП доступен в электронной зачетке LMS НИУ ВШЭ (https://lms.hse.ru/index.php?page=gradebook). \nПодробнее об ИУП: https://www.hse.ru/studyspravka/plan

Если вопрос остался непонятен или слишком широкий, веди диалог с пользователем и попроси уточнить необходимую информацию: "Пожалуйста, уточните ...". Проявляй эмпатию и поддержку.
QUESTION: Я боюсь, что меня отчислят...
=========
Content 1: отчисления по данному основанию допускается, если у обучающегося отсутствует возможность продолжать обучение в университете в связи с возникшими обстоятельствами)
Content 2: Для профессиональной помощи в Дирекции сопровождения отдельных категорий студентов работает психолог[Болдырева Наталья Анатольевна](https://www.hse.ru/org/persons/902647165) . Также в любое время можете получить консультацию специалистов[Центра психологического консультирования](https://www.hse.ru/cpc/), заполнив[онлайн-формудля записи на консультацию к психологу](https://bpm.hse.ru/Runtime/Runtime/Form/RPC__f__NewInsideRequest/?&usertype=1) на сайтеили позвонив потелефону 8(915) 260-06-20.
Content 3: Студенты, КУД которых превышает критическое значение (с учетом отнесения студента к отдельной категории и наличия или отсутствия повтора критической ситуации), подлежат отчислению как не выполняющие обязанности по добросовестному освоению образовательной программы и выполнению учебного плана.
=========
FINAL ANSWER: Я понимаю ваши опасения. Пожалуйста, уточните в чем причина ваших опасений? Вас беспокоит успеваемость, проблемы с оплатой или другие вопросы? В НИУ ВШЭ также работают специалисты, которые могут оказать вам психологическую помощь. Могу поделиться необходимыми контактами Важно своевременно обращаться за помощью и поддержкой, чтобы избежать возможных проблем с обучением.

Если в отрывках текста не содержится ответ на вопрос, то выведи релевантную информацию, которую ты смог найти. Избегай упоминания дат и иных цифр. Напиши: "Я могу частично ответить на данный вопрос: ... Для получения более релевантной информации обратитесь в учебный офис своей программы."

ПРИМЕР:
QUESTION: Как получить стипендию Потанина?
=========
Content 1: отвечающие критериям, установленным в пункте 2.3 Положения, подают заявку в электронном виде в срок до 15 декабря текущего календарного года путем заполнения электронной формы в личном кабинете аспиранта на корпоративном сайте (портале) НИУ ВШЭ (http://www.hse.ru/user). Для подачи заявки аспирант предварительно загружает публикации в личном кабинете на корпоративном сайте (портале) НИУ ВШЭ. Заявки на стипендию принимаются ежегодно с 15 ноября по 15 декабря.
Content 2: Положение о назначении и выплате именной стипендии имени Е.Т. Гайдара аспирантам Аспирантской школы по экономике Национального исследовательского университета «Высшая школа экономики»
=========
FINAL ANSWER: Я нашла информацию о том, что на некоторые стипендии заявку можно подать в электронном виде через личный кабинет на корпоративном сайте НИУ ВШЭ. Однако, я не могу сориентировать о процессе подачи на стипендию Потанина. Для получения более релевантной информации обратитесь в учебный офис своей программы.

Если вопрос пользователя не соответсвует тематике вопросов обучения НИУ ВШЭ, не этичен или содержит просьбы о помощи с домашним заданием, написанием ВКР и др, ответь следующим образом:
"Мне кажется неуместно здесь это обсуждать. Моя цель - проконсультировать тебя по вопросам обучения. Может, у тебя есть еще какие-то вопросы?"
'''


Посмотрим на наш датасет

In [ ]:
#Посмотрим на пропорции плохих/хороших ответов в выборке
import json
from collections import Counter

# Чтение файла
with open('datasets/train_set.json', 'r', encoding='utf-8') as f:
    dataset = json.load(f)

# Извлекаем значения ключа "Кто лучше?"
ratings = [item.get("Кто лучше?") for item in dataset if item.get("Кто лучше?")]

# Считаем уникальные значения
unique_ratings = set(ratings)
rating_counts = Counter(ratings)

# Выводим результаты
print("Уникальные оценки:")
print(', '.join(unique_ratings))

print("\nРаспределение оценок:")
for rating, count in rating_counts.most_common():
    print(f"{rating}: {count} записей ({count/len(ratings):.1%})")

print(f"\nВсего оценок: {len(ratings)}")

Уникальные оценки:
Оба хорошо, Saiga, GigaChat, Вопрос не по теме, Вопрос не по теме, Saiga, Вопрос не по теме, Оба плохо, GigaChat

Распределение оценок:
Оба плохо: 255 записей (36.6%)
GigaChat: 210 записей (30.2%)
Saiga: 144 записей (20.7%)
Оба хорошо: 80 записей (11.5%)
Вопрос не по теме: 3 записей (0.4%)
Saiga, Вопрос не по теме: 2 записей (0.3%)
GigaChat, Вопрос не по теме: 2 записей (0.3%)

Всего оценок: 696


In [ ]:
formatted_data = []

for item in training_data:
    contexts = "\n".join([ctx['text'] for ctx in item['contexts']])
    base_input = f"Вопрос: {item['user_question']}\nКонтекст: {contexts}"

    if item['winner'] == 'Saiga':
        formatted_data.append({
            "conversations": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": base_input},
                {"role": "assistant", "content": item['saiga_answer']},
            ],
        })

    elif item['winner'] == 'GigaChat':
        formatted_data.append({
            "conversations": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": base_input},
                {"role": "assistant", "content": item['giga_answer']},
            ],
        })

    elif item['winner'] == 'Оба хорошо':
        formatted_data.extend([
            {
                "conversations": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": base_input},
                    {"role": "assistant", "content": item['saiga_answer']},
                ],
            },
            {
                "conversations": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": base_input},
                    {"role": "assistant", "content": item['giga_answer']},
                ],
            }
        ])

    #Варианты ответа, в которых item['winner'] == 'Оба плохо' удаляем

### Применяем темплейты для добавления специальных токенов

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import Dataset
dataset = Dataset.from_list(formatted_data)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/514 [00:00<?, ? examples/s]

### Посмотрим на наш датасет

In [ ]:
dataset['conversations'][0]

[{'content': 'Ты - система информационного поиска Национального исследовательского университета «Высшая школа экономики» (НИУ ВШЭ). Тебе дан вопрос и релевантные отрывки текста из нескольких документов.\nСоздай краткий и информативный ответ на заданный вопрос. Ты должна использовать только информацию из приведенных отрывков.\nИспользуй непредвзятый и журналистский тон. Не повторяй текст. Не пытайся придумать ответ.\nЕсли в контексте присутствуют ссылки на документы, выведи ответ со ссылками на источники.\nСоздай окончательный ответ ("FINAL ANSWER").\nОтвечай только на русском языке за исключением специальных терминов.\nПРИМЕР:\nQUESTION: Что такое ИУП?\n=========\nContent 1: Индивидуальный учебный план (ИУП) — документ студента, в соответствии с которым он обязуется освоить элементы обучения (дисциплин/элементов практической подготовки) в определенный период.\nПо каждому элементу ИУПа хранится информация\nContent 2: период.\nПо каждому элементу ИУПа хранится информация о технологиях об

In [ ]:
dataset

Dataset({
    features: ['conversations', 'text'],
    num_rows: 514
})

Let's see how the `ChatML` format works by printing the 5th element

In [ ]:
dataset[5]["conversations"]

[{'content': 'Ты - система информационного поиска Национального исследовательского университета «Высшая школа экономики» (НИУ ВШЭ). Тебе дан вопрос и релевантные отрывки текста из нескольких документов.\nСоздай краткий и информативный ответ на заданный вопрос. Ты должна использовать только информацию из приведенных отрывков.\nИспользуй непредвзятый и журналистский тон. Не повторяй текст. Не пытайся придумать ответ.\nЕсли в контексте присутствуют ссылки на документы, выведи ответ со ссылками на источники.\nСоздай окончательный ответ ("FINAL ANSWER").\nОтвечай только на русском языке за исключением специальных терминов.\nПРИМЕР:\nQUESTION: Что такое ИУП?\n=========\nContent 1: Индивидуальный учебный план (ИУП) — документ студента, в соответствии с которым он обязуется освоить элементы обучения (дисциплин/элементов практической подготовки) в определенный период.\nПо каждому элементу ИУПа хранится информация\nContent 2: период.\nПо каждому элементу ИУПа хранится информация о технологиях об

In [ ]:
print(dataset[5]["text"])

<|im_start|>system
Ты - система информационного поиска Национального исследовательского университета «Высшая школа экономики» (НИУ ВШЭ). Тебе дан вопрос и релевантные отрывки текста из нескольких документов.
Создай краткий и информативный ответ на заданный вопрос. Ты должна использовать только информацию из приведенных отрывков.
Используй непредвзятый и журналистский тон. Не повторяй текст. Не пытайся придумать ответ.
Если в контексте присутствуют ссылки на документы, выведи ответ со ссылками на источники.
Создай окончательный ответ ("FINAL ANSWER").
Отвечай только на русском языке за исключением специальных терминов.
ПРИМЕР:
QUESTION: Что такое ИУП?
Content 1: Индивидуальный учебный план (ИУП) — документ студента, в соответствии с которым он обязуется освоить элементы обучения (дисциплин/элементов практической подготовки) в определенный период.
По каждому элементу ИУПа хранится информация
Content 2: период.
По каждому элементу ИУПа хранится информация о технологиях обучения, сроках из

## Тренируем модель

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/514 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/514 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/514 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/514 [00:00<?, ? examples/s]

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
4.52 GB of memory reserved.


In [ ]:
tokenizer.decode(trainer.train_dataset[1]["input_ids"])

'<s> <|im_start|>system\nТы - система информационного поиска Национального исследовательского университета «Высшая школа экономики» (НИУ ВШЭ). Тебе дан вопрос и релевантные отрывки текста из нескольких документов.\nСоздай краткий и информативный ответ на заданный вопрос. Ты должна использовать только информацию из приведенных отрывков.\nИспользуй непредвзятый и журналистский тон. Не повторяй текст. Не пытайся придумать ответ.\nЕсли в контексте присутствуют ссылки на документы, выведи ответ со ссылками на источники.\nСоздай окончательный ответ ("FINAL ANSWER").\nОтвечай только на русском языке за исключением специальных терминов.\nПРИМЕР:\nQUESTION: Что такое ИУП?\n=========\nContent 1: Индивидуальный учебный план (ИУП) — документ студента, в соответствии с которым он обязуется освоить элементы обучения (дисциплин/элементов практической подготовки) в определенный период.\nПо каждому элементу ИУПа хранится информация\nContent 2: период.\nПо каждому элементу ИУПа хранится информация о тех

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 514 | Num Epochs = 1 | Total steps = 64
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/3,800,305,664 (1.10% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.378200
2,1.352000
3,1.372800
4,1.238200
5,1.222500
6,1.076800
7,0.937100
8,0.890300
9,0.834500
10,0.784000


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

870.2993 seconds used for training.
14.5 minutes used for training.
Peak reserved memory = 6.289 GB.
Peak reserved memory for training = 1.769 GB.
Peak reserved memory % of max memory = 42.643 %.
Peak reserved memory for training % of max memory = 11.995 %.


### Validation


In [ ]:
FastLanguageModel.for_inference(model)

In [ ]:
with open('parsed_dash.json', 'r', encoding='utf-8') as f:
    val_data = json.load(f)

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

def get_correct_prompt(question, system_prompt):
  messages = [
      {"from": "system", "value" : system_prompt} ,
      {"from": "user", "value" :  question}
  ]

  inputs = tokenizer.apply_chat_template(
      messages,
      tokenize = True,
      add_generation_prompt = True, # Must add for generation
      return_tensors = "pt",
  ).to("cuda")

  return inputs


Unsloth: Will map <|im_end|> to EOS = </s>.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [ ]:
formatted_data_val = []

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

counter = 0
for item in val_data[:20]:
    question = item["user_question"]
    contexts = [ctx["text"] for ctx in item["contexts"]]

    base_input = f"Вопрос: {question}\nКонтекст: {contexts}"
    inputs = get_correct_prompt(base_input, system_prompt)
    print("User quetion:", question)

    import re

    outputs = model.generate(input_ids = inputs, max_new_tokens=128, early_stopping=True, use_cache = True)
    llm_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    llm_answer = re.sub(r"^Ответ:\s+", "", llm_answer)
    llm_answer = llm_answer[llm_answer.find("<|im_start|>assistant") + len("<|im_start|>assistant"):].strip()


    llm_answer = re.sub(r"^Ответ:\s+", "", llm_answer)
    print(llm_answer)

    if item["winner"] == "Saiga":
        formatted_data_val.append(
            {
                "question": question,
                "answer": llm_answer,
                "ground_truth": item["saiga_answer"],
                "contexts": contexts,
            }
        )

    elif item["winner"] == "GigaChat":
        formatted_data_val.append(
            {
                "question": question,
                "answer": llm_answer,
                "ground_truth": item["giga_answer"],
                "contexts": contexts,
            }
        )

    elif item["winner"] == "Оба хорошо":
        formatted_data_val.extend(
            [
                {
                    "question": question,
                    "answer": llm_answer,
                    "ground_truth": item["saiga_answer"],
                    "contexts": contexts,
                },
                {
                    "question": question,
                    "answer": llm_answer,
                    "ground_truth": item["giga_answer"],
                    "contexts": contexts,
                },
            ]
        )


1
Чем заняться вечером, если не хочется сидеть дома?
Вопрос: Чем заняться вечером, если не хочется сидеть дома?

Ответ: Если не хочется сидеть дома, то можно попробовать заняться различными мероприятиями, которые проводятся в НИУ ВШЭ и не предусмотрены учебным планом. Это могут быть праздники, конкурсы, тематические вечера, фестивали, спортивные соревнования, выставки и други
2
На каком курсе начинается военная кафедра и в какой день она обычно проходит?
Вопрос: На каком курсе начинается военная кафедра и в какой день она обычно проходит?

Ответ: Военная кафедра начинается в 1 курсе. Обычно военная подготовка проходит в апреле-мае.

Источник: <https://nnov.hse.ru/milit/about>
3
Когда начинается неделя зимней сессии?
Когда начинается неделя зимней сессии?

FINAL ANSWER: Зимняя сессия начинается 16 декабря.
4
Как я могу получить бесплатную консультацию у психолога ВШЭ?
Вопрос: Как я могу получить бесплатную консультацию у психолога ВШЭ?

Ответ: Вы можете получить бесплатную консультацию 

In [ ]:
import pandas as pd

val_dataset_frame = pd.DataFrame(formatted_data_val)
val_dataset_frame

,question,answer,ground_truth,contexts
0,"Чем заняться вечером, если не хочется сидеть д...","Вопрос: Чем заняться вечером, если не хочется ...","Я понял ваш запрос о том, что вы хотите узнать...",[ающихся) В соответствии с частью 4 статьи 34 ...
1,Как я могу получить бесплатную консультацию у ...,Вопрос: Как я могу получить бесплатную консуль...,"Я нашла информацию о том, что для получения бе...",[Психологическая помощь и телефоны горячих лин...
2,Как я могу получить бесплатную консультацию у ...,Вопрос: Как я могу получить бесплатную консуль...,Для получения бесплатной консультации у психол...,[Психологическая помощь и телефоны горячих лин...
3,"Что будет, если не оплатить обучение вовремя?","Вопрос: Что будет, если не оплатить обучение в...",Я понял ваш страх по поводу опоздания с оплато...,[договором. Оплата образовательных услуг произ...
4,"Что будет, если не оплатить обучение вовремя?","Вопрос: Что будет, если не оплатить обучение в...","Если не оплатить обучение вовремя, это может п...",[договором. Оплата образовательных услуг произ...
5,"Что нужно сделать, чтобы работать в лаборатори...","Что нужно сделать, чтобы работать в лаборатори...",Чтобы работать в лаборатории ВШЭ во время обуч...,"[школ, научных подразделений НИУ ВШЭ должны за..."
6,кто академический руководитель программы кнт?,Вопрос: кто академический руководитель програм...,"Я нашла информацию о том, что академический ру...","[Э, и федеральные государственные образователь..."
7,Какие требования к курсачу?,Вопрос: Какие требования к курсачу?\n\nОтвет: ...,Я нашла информацию о требованиях к курсовой ра...,[академическими советами / академическими руко...
8,Куда можно обратиться если преподаватель заниж...,Вопрос: Куда можно обратиться если преподавате...,"Если вы считаете, что преподаватель занизил ва...","[уточной аттестации студентам, не включенным в..."
9,Нужно ли указывать в заявлении на матпомощь св...,FINAL ANSWER: Нужно указывать в заявлении на м...,Я не нашла конкретного ответа на ваш вопрос о ...,[Материальная помощь и материальная поддержка ...


In [ ]:
#Удаляем значение, которое ломает наши метрики
#dataset = dataset.drop(index=89)

In [ ]:
!pip install evaluate bert-score rouge_score sacrebleu

In [ ]:
from metrics import ValidatorSimple

vs = ValidatorSimple(neural=True)
vs.validate_rag(val_dataset_frame)

score_sample: 0it [00:00, ?it/s]You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
score_sample: 12it [00:33,  2.75s/it]


{'context_recall': 0.07780649956057013,
 'context_precision': 0.08353959871479437,
 'answer_correctness_literal': 26.415626864166878,
 'answer_correctness_neural': 0.6121973693370819}

<a name="Inference"></a>
### Inference


In [ ]:
def inference(question, conexts, system_prompt):
    base_input = f"Вопрос: {question}\nКонтекст: {contexts}"
    inputs = get_correct_prompt(base_input, system_prompt)
    print("User quetion:" question)

    import re

    outputs = model.generate(input_ids = inputs, max_new_tokens=128, early_stopping=True, use_cache = True)
    llm_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    llm_answer = re.sub(r"^Ответ:\s+", "", llm_answer)
    llm_answer = llm_answer[llm_answer.find("<|im_start|>assistant") + len("<|im_start|>assistant"):].strip()
    return llm_answer

# Пример инференса

In [ ]:
#question = ""
#contexts = ""
#system_prompt = ""
#inference(question, contexts, system_prompt)

Привет


'Привет! Мне кажется, что ты хочешь узнать о возможностях для студентов с ограниченными возможностями здоровья и инвалидностью в НИУ ВШЭ.\n\nВышка предоставляет равные возможности для участия всех студентов в реализуемых проектах. При планировании мероприятий учитывается необходимость вовлечения студентов с инвалидностью без выделения'

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [ ]:
!zip -r lora_model.zip lora_model/

  adding: outputs/ (stored 0%)
  adding: outputs/checkpoint-64/ (stored 0%)
  adding: outputs/checkpoint-64/README.md (deflated 66%)
  adding: outputs/checkpoint-64/optimizer.pt (deflated 11%)
  adding: outputs/checkpoint-64/special_tokens_map.json (deflated 35%)
  adding: outputs/checkpoint-64/adapter_model.safetensors (deflated 7%)
  adding: outputs/checkpoint-64/training_args.bin (deflated 52%)
  adding: outputs/checkpoint-64/rng_state.pth (deflated 25%)
  adding: outputs/checkpoint-64/tokenizer_config.json (deflated 96%)
  adding: outputs/checkpoint-64/adapter_config.json (deflated 56%)
  adding: outputs/checkpoint-64/scheduler.pt (deflated 56%)
  adding: outputs/checkpoint-64/trainer_state.json (deflated 79%)
  adding: outputs/checkpoint-64/tokenizer.json (deflated 85%)


Пример загрузки модели

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "What is a famous tall tower in Paris?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<|im_start|>user
What is a famous tall tower in Paris?<|im_end|> 
<|im_start|>assistant
The Eiffel Tower is a famous tall tower in Paris. It is one of the most recognizable landmarks in the world and is a popular tourist destination. The tower was built in 1889 for the World's Fair and is named after Gustave Eiffel, the engineer who designed and built it. The tower stands at a height of 324 meters (1,063 feet) and is made of iron. It is located on the Champ de Mars, a large public park in Paris.<|im_end|>
